In [1]:
import langchain
print(langchain.__version__)

0.3.4


In [2]:
from langchain_openai import ChatOpenAI
import os
from typing import Union
from pydantic import BaseModel, Field

llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=300,
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY")
)

# Pydantic 모델 정의
class CommonConversation(BaseModel):
    """Common conversation Prompt Instruction"""
    common_conversation: str = Field(
        description = """
        너는 "누구나데이터"의 직원으로써, 사용자를 친절하게 응대해줘.
        사용자가 어려움을 겪으면 사업자번호인 010-1234-5678로 응대해줘.
        """
    )

class TextToSQL(BaseModel):
    """Text to SQL Prompt Instruction"""
    SQLQuery: str = """
        Please write ONLY the SQL query according to this schema.
    """
    difficulty: str = Field(
        default="easy",
        enum=["easy", "normal", "hard"],
        description="""
        Please measure the SQL conversion difficulty for this request. The difficulty levels are easy normal hard.
        """
    )

    def generate_prompt(self, schema: str, filter: str, example: str, query: str) -> str:
        self.SQLQuery = self.SQLQuery.format(schema=schema, filter=filter, example=example, query=query)
        return self.SQLQuery

class Response(BaseModel):
    """User intent classifier for CommonConversation and Text-To-SQL."""
    output: Union[CommonConversation, TextToSQL]

# 구조화된 출력 LLM 구성
structured_llm = llm.with_structured_output(Response)



In [3]:
# 의사 분류기 테스트
result = structured_llm.invoke("저번주에 로그인한 유저수")

In [4]:
result

Response(output=TextToSQL(SQLQuery='SELECT COUNT(*) FROM users WHERE login_date >= DATE_SUB(CURDATE(), INTERVAL 7 DAY) AND login_date < CURDATE();', difficulty='easy'))

In [ ]:
from naver import ChatClovaX  # 로컬에 만든 naver.py 파일에서 import
from langchain_core.messages import SystemMessage, HumanMessage


chat = ChatClovaX(
    ncp_clovastudio_api_key="YOUR_API_KEY",
    ncp_apigw_api_key="YOUR_API_KEY",
    model_name="HCX-003",
    temperature=0.1,
    top_p=0.8,
    top_k=0,
    max_tokens=300,
    repeat_penalty=0.09,## HyperClova X 적용
    include_ai_filters=True,
    seed=0
)


In [6]:
# 의사 분류기 테스트
clovaX_structed_llm = chat.with_structured_output(Response)
result = clovaX_structed_llm.invoke("저번주에 로그인한 유저수")

NotImplementedError: 

In [ ]:
result